<a href="https://colab.research.google.com/github/subhayu99/AiNotebooks/blob/master/DPHI_Deep_Learning_Assignment_on_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
 import tensorflow as tf
print(tf.__version__)
from tensorflow import keras

2.3.0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
 
#Split them into train & test
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
print("x_train.shape:", x_train[0].shape)
print("y_train.shape", y_train.shape)

x_train.shape: (32, 32, 3)
y_train.shape (50000,)


In [46]:
test_new = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/cifar_image_flattened_pixels.csv")
new_test_data = (test_new.to_numpy())/255.0
new_test_data = new_test_data.reshape(2000, 32, 32, 3)
print(test_new.shape)
print("new_test_data.shape", new_test_data.shape)
# print(new_test_data[0])

(2000, 3072)
new_test_data.shape (2000, 32, 32, 3)


In [ ]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


In [ ]:
i = Input(shape=x_train[0].shape)
 
x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
 
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(K, activation='softmax')(x)
 
model = Model(i, x)

In [ ]:
# Note: make sure you are using the GPU for this.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [133]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("my_model")
 
# Let's check:
np.testing.assert_allclose(
    model.predict(x_test), reconstructed_model.predict(x_test)
)
 
# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50)
 
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model")

KeyboardInterrupt: ignored

In [107]:
predictions = model.predict(new_test_data)

In [108]:
print(predictions[2])

[0.0000001  0.         0.00000005 0.00000002 0.08819173 0.01505479
 0.         0.8967533  0.         0.        ]


In [77]:
np.set_printoptions(suppress=True)

In [121]:
predictions = predictions.reshape(2000, 10)
predictions = np.argmax(predictions, axis=1)
print(predictions.shape)
print(res)

(2000,)
   0     1     2     3     4     5     ...  1994  1995  1996  1997  1998  1999
0     2     3     7     8     6     1  ...     9     8     0     3     3     4

[1 rows x 2000 columns]


In [119]:
# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(predictions.reshape(1, 2000)) #preditcions are nothing but the final predictions of your model on input features of your new unseen test data
res.index = test_new.index # its important for comparison. Here "test_new" is your new test dataset
res.columns = ["prediction"]
 
# To download the csv file locally
from google.colab import files
res.to_csv('prediction_results.csv')         
files.download('prediction_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Accuracy on new_test dataset was 82.3%

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>